In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

dataset_path = r"D:\symbi\Semester-3\AI\project\Data\images"

# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

#train and Validation
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Model Architecture
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Gradually Unfreeze Layers
for layer in base_model.layers[:-20]:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=80,
    callbacks=[early_stop]
)

# Save the Model
model.save("waste_classifier_model.h5")

Found 12000 images belonging to 30 classes.
Found 3000 images belonging to 30 classes.


C:\Users\Sohan Kanse\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/80
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2962 - loss: 2.9502

C:\Users\Sohan Kanse\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


375/375 ━━━━━━━━━━━━━━━━━━━━ 707s 2s/step - accuracy: 0.2966 - loss: 2.9485 - val_accuracy: 0.7693 - val_loss: 1.0528
Epoch 2/80
375/375 ━━━━━━━━━━━━━━━━━━━━ 480s 1s/step - accuracy: 0.6569 - loss: 1.4920 - val_accuracy: 0.8113 - val_loss: 0.8823
Epoch 3/80
375/375 ━━━━━━━━━━━━━━━━━━━━ 543s 1s/step - accuracy: 0.7244 - loss: 1.2330 - val_accuracy: 0.8383 - val_loss: 0.8242
Epoch 4/80
375/375 ━━━━━━━━━━━━━━━━━━━━ 573s 2s/step - accuracy: 0.7567 - loss: 1.1087 - val_accuracy: 0.8500 - val_loss: 0.7831
Epoch 5/80
375/375 ━━━━━━━━━━━━━━━━━━━━ 532s 1s/step - accuracy: 0.7830 - loss: 1.0229 - val_accuracy: 0.8643 - val_loss: 0.7494
Epoch 6/80
375/375 ━━━━━━━━━━━━━━━━━━━━ 478s 1s/step - accuracy: 0.8144 - loss: 0.9049 - val_accuracy: 0.8653 - val_loss: 0.7248
Epoch 7/80
375/375 ━━━━━━━━━━━━━━━━━━━━ 475s 1s/step - accuracy: 0.8321 - loss: 0.8467 - val_accuracy: 0.8737 - val_loss: 0.7231
Epoch 8/80
375/375 ━━━━━━━━━━━━━━━━━━━━ 564s 2s/step - accuracy: 0.8376 - loss: 0.8251 - val_accuracy: 0.870

In [4]:
# Evaluate the Model on Validation Data (used as test data in this case)
val_loss, val_accuracy = model.evaluate(val_generator)

# Print the Results
print(f"Validation/Test Accuracy: {val_accuracy * 100:.2f}%")
print(f"Validation/Test Loss: {val_loss:.4f}")


94/94 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.8615 - loss: 0.8492
Validation/Test Accuracy: 86.30%
Validation/Test Loss: 0.8265


In [6]:
save_path = r"D:\symbi\Semester-3\AI\project\models\waste_classifier_new.keras"
model.save(save_path)  # Save using .keras extension
print(f"Model saved successfully at: {save_path}")


Model saved successfully at: D:\symbi\Semester-3\AI\project\models\waste_classifier_new.keras


In [7]:
from tensorflow.keras.models import load_model
save_path = r"D:\symbi\Semester-3\AI\project\models\waste_classifier_new.keras"
# Load the saved model
loaded_model = load_model(save_path)

# Verify it works
loaded_model.summary()


C:\Users\Sohan Kanse\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 48 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 62720)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     8,028,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         3,870 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,847,967 (117.68 MB)

 Trainable params: 9,383,118 (35.79 MB)

 Non-trainable params: 2,698,611 (10.29 MB)

 Optimizer params: 18,766,238 (71.59 MB)